# Gemma3-1B GRPO Training Notebook

This notebook trains Gemma3-1B with GRPO (Group Relative Policy Optimization) for improved reasoning.

**Requirements:**
- Google Colab with TPU runtime (recommended) or GPU
- HuggingFace account with Gemma access

**Output:**
- LoRA checkpoint files that can be downloaded and used locally

## 1. Setup

In [ ]:
# Install dependencies
!pip install -q kagglehub
!pip install -q datasets
!pip install -q wandb
!pip install -q "numpy<2.0"
!pip install git+https://github.com/google/tunix.git
!pip uninstall -q -y flax
!pip install flax==0.12.0
!pip install -q 'transformers<=4.57.1'

print("\n" + "="*60)
print("Installation complete!")
print("="*60)

In [ ]:
# Configuration
# =============================================================================
# EDIT THESE VALUES
# =============================================================================

# Training settings
NUM_BATCHES = 500          # Number of training batches (500 = ~30 min on TPU)
LEARNING_RATE = 3e-6       # Learning rate
LORA_RANK = 64             # LoRA rank
LORA_ALPHA = 64.0          # LoRA alpha

# Dataset settings
USE_OPENRUBRICS = True     # Use OpenRubrics dataset
OPENRUBRICS_MAX = 2000     # Max examples from OpenRubrics

# Checkpoint settings
SAVE_TO_DRIVE = False       # Save checkpoints to Google Drive
EXPERIMENT_NAME = "gemma3_grpo_reasoning"

# =============================================================================
# CREDENTIALS - Uses Colab Secrets or Kaggle Secrets
# =============================================================================
import os

# Try Google Colab secrets first
try:
    from google.colab import userdata
    os.environ['WANDB_API_KEY'] = userdata.get('WANDB_API_KEY')
    os.environ['KAGGLE_USERNAME'] = userdata.get('KAGGLE_USERNAME')
    os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')
    print("Using Google Colab secrets")
except (ImportError, ModuleNotFoundError):
    # Fall back to Kaggle secrets
    try:
        from kaggle_secrets import UserSecretsClient
        secrets = UserSecretsClient()
        os.environ['WANDB_API_KEY'] = secrets.get_secret('WANDB_API_KEY')
        os.environ['KAGGLE_USERNAME'] = secrets.get_secret('KAGGLE_USERNAME')
        os.environ['KAGGLE_KEY'] = secrets.get_secret('KAGGLE_KEY')
        print("Using Kaggle secrets")
    except (ImportError, ModuleNotFoundError):
        print("WARNING: No secrets provider found. Set environment variables manually:")
        print("  export WANDB_API_KEY=your_key")
        print("  export KAGGLE_USERNAME=your_username")
        print("  export KAGGLE_KEY=your_key")

print(f"\nTraining config:")
print(f"  Batches: {NUM_BATCHES}")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"  LoRA rank: {LORA_RANK}")
print(f"  Kaggle user: {os.environ.get('KAGGLE_USERNAME', 'not set')}")

In [ ]:
# Mount Google Drive (optional but recommended)
if SAVE_TO_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
    CHECKPOINT_DIR = f"/content/drive/MyDrive/{EXPERIMENT_NAME}/checkpoints"
    import os
    os.makedirs(CHECKPOINT_DIR, exist_ok=True)
    print(f"Checkpoints will be saved to: {CHECKPOINT_DIR}")
else:
    CHECKPOINT_DIR = "/content/checkpoints"
    import os
    os.makedirs(CHECKPOINT_DIR, exist_ok=True)

In [ ]:
# Imports
import functools
import gc
import os
import re
import csv
import shutil
from pprint import pprint
from flax import nnx
import grain
import humanize
import jax
import jax.numpy as jnp
import kagglehub
import optax
from orbax import checkpoint as ocp
from pathlib import Path
import qwix
from tqdm.auto import tqdm
from tunix.generate import sampler as sampler_lib
from tunix.generate import tokenizer_adapter as tokenizer_lib
from tunix.models.gemma3 import params
from tunix.models.gemma3 import model
from tunix.rl import rl_cluster as rl_cluster_lib
from tunix.rl.grpo.grpo_learner import GRPOConfig, GRPOLearner
from tunix.rl.rollout import base_rollout
from tunix.sft import metrics_logger
from datasets import load_dataset

print(f"JAX devices: {jax.devices()}")

## 2. Prompt Template

In [ ]:
# Prompt configuration
REASONING_START = "<reasoning>"
REASONING_END = "</reasoning>"
SOLUTION_START = "<answer>"
SOLUTION_END = "</answer>"

SYSTEM_PROMPT = f"""You are given a problem. Think carefully and show your detailed reasoning step-by-step. Place your reasoning between {REASONING_START} and {REASONING_END}. After completing your reasoning, provide the final answer between {SOLUTION_START} and {SOLUTION_END}."""

TEMPLATE = """<start_of_turn>user
{system_prompt}

{question}<end_of_turn>
<start_of_turn>model"""

def format_prompt(question, rubric=None):
    rubric_block = f"\nRubric:\n{rubric}\n\n" if rubric else ""
    return TEMPLATE.format(
        system_prompt=SYSTEM_PROMPT,
        question=f"{rubric_block}{question}",
    )

print("Prompt template configured.")

## 3. Load Dataset

In [ ]:
def load_openrubrics_dataset(max_examples=2000):
    """Load OpenRubrics dataset from HuggingFace."""
    print("Loading OpenRubrics dataset...")
    try:
        raw_ds = load_dataset("OpenRubrics/OpenRubrics", split="train")
    except Exception as e:
        print(f"Error loading OpenRubrics: {e}")
        return []
    
    columns = list(raw_ds.column_names)
    
    def pick_col(options):
        for name in options:
            if name in columns:
                return name
        return ""
    
    question_col = pick_col(["instruction", "prompt", "question", "input"])
    rubric_col = pick_col(["rubric", "scoring_rubric", "criteria"])
    response_col = pick_col(["response", "model_answer", "answer", "output"])
    
    data = []
    for row in raw_ds:
        question = row.get(question_col, "")
        if not question:
            continue
        
        rubric = row.get(rubric_col, "")
        if isinstance(rubric, dict):
            rubric = "\n".join(f"{k}: {v}" for k, v in rubric.items() if v)
        elif isinstance(rubric, list):
            rubric = "\n".join(str(r) for r in rubric if r)
        
        data.append({
            "question": str(question),
            "rubric": str(rubric) if rubric else "",
            "reference_response": str(row.get(response_col, "")),
        })
        
        if len(data) >= max_examples:
            break
    
    print(f"Loaded {len(data)} examples from OpenRubrics")
    return data

# Load dataset
if USE_OPENRUBRICS:
    train_data = load_openrubrics_dataset(max_examples=OPENRUBRICS_MAX)
else:
    train_data = []  # Add GSM8K loading if needed

print(f"\nTotal training examples: {len(train_data)}")

In [ ]:
# Create grain dataset
def create_dataset(data):
    return (
        grain.MapDataset.source(data)
        .shuffle(seed=42)
        .map(
            lambda x: {
                "prompts": format_prompt(x["question"], x.get("rubric")),
                "question": x["question"],
                "rubric": x.get("rubric", ""),
                "reference_response": x.get("reference_response", ""),
            }
        )
    )

# Split data
split_idx = int(len(train_data) * 0.9)
train_split = train_data[:split_idx]
test_split = train_data[split_idx:]

train_dataset = create_dataset(train_split)
test_dataset = create_dataset(test_split)

print(f"Train: {len(train_split)}, Test: {len(test_split)}")

## 4. Load Model

In [ ]:
# Download Gemma3-1B checkpoint
# Use Kaggle API directly to avoid Colab's proxy issues
import json
import subprocess

# Write credentials to kaggle.json (from environment variables set by secrets)
kaggle_creds = {
    "username": os.environ.get('KAGGLE_USERNAME', ''),
    "key": os.environ.get('KAGGLE_KEY', '')
}

if not kaggle_creds['username'] or not kaggle_creds['key']:
    raise ValueError("KAGGLE_USERNAME and KAGGLE_KEY must be set. Check your Colab/Kaggle secrets.")

os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)
kaggle_json_path = os.path.expanduser("~/.kaggle/kaggle.json")
with open(kaggle_json_path, "w") as f:
    json.dump(kaggle_creds, f)
os.chmod(kaggle_json_path, 0o600)

print(f"Authenticated as: {kaggle_creds['username']}")

# Force kagglehub to not use Colab backend by patching before download
import kagglehub.colab_cache_resolver
kagglehub.colab_cache_resolver._COLAB_BACKEND_AVAILABLE = False

# Also patch the env check
import kagglehub.env
kagglehub.env.is_colab_env = lambda: False

# Clear any cached resolvers
import importlib
import kagglehub.models
importlib.reload(kagglehub.models)

import kagglehub
MODEL_CP_PATH = kagglehub.model_download("google/gemma-3/flax/gemma3-1b-it")
print(f"Model path: {MODEL_CP_PATH}")

In [ ]:
# Create model config
config = model.ModelConfig.gemma3_1b()
gemma = params.create_model_from_checkpoint(MODEL_CP_PATH, config)
tokenizer = params.create_tokenizer()

# Save initial state
INTERMEDIATE_CKPT_DIR = "/tmp/intermediate_ckpt/"
_, state = nnx.split(gemma)
checkpointer = ocp.StandardCheckpointer()
checkpointer.save(INTERMEDIATE_CKPT_DIR + "state", state)

del gemma
gc.collect()
print("Base model state saved.")

In [ ]:
# Load reference model
MESH = [(1, 1), ("fsdp", "tp")]

def get_gemma_ref_model(ckpt_path):
    model_config = model.ModelConfig.gemma3_1b()
    abs_gemma = nnx.eval_shape(
        lambda: params.create_model_from_checkpoint(MODEL_CP_PATH, model_config)
    )
    
    abs_state = nnx.state(abs_gemma)
    mesh = qwix.create_mesh(MESH)
    sharding_rules = gemma.default_sharding_config(model_config, mesh)
    shardings = qwix.make_shardings(abs_state, sharding_rules)
    
    checkpointer = ocp.StandardCheckpointer()
    restored_params = checkpointer.restore(ckpt_path, target=abs_state)
    
    graph_def, _ = nnx.split(abs_gemma)
    gemma_model = nnx.merge(graph_def, restored_params)
    return gemma_model, mesh, model_config

ref_model, mesh, model_config = get_gemma_ref_model(
    ckpt_path=os.path.join(INTERMEDIATE_CKPT_DIR, "state")
)
print("Reference model loaded.")

In [ ]:
# Create LoRA model
def get_lora_model(base_model, mesh):
    lora_provider = qwix.LoRAProvider(
        rank=LORA_RANK,
        alpha=LORA_ALPHA,
        include=[r".*attn.*"],
        exclude=[r".*embed.*"],
    )
    
    model_input = base_model.get_model_input()
    lora_model = qwix.apply_lora_to_model(
        base_model, lora_provider, **model_input
    )
    
    with mesh:
        state = nnx.state(lora_model)
        sharded_state = jax.tree.map(jax.lax.with_sharding_constraint, state, state)
        nnx.update(lora_model, sharded_state)
    
    return lora_model

lora_policy = get_lora_model(ref_model, mesh=mesh)
print(f"LoRA model created with rank={LORA_RANK}, alpha={LORA_ALPHA}")

## 5. Reward Functions

In [ ]:
import string
import difflib
from collections import Counter

# Format matching regex
match_format = re.compile(
    rf"{REASONING_START}.*?{REASONING_END}.*?{SOLUTION_START}.*?{SOLUTION_END}",
    re.DOTALL
)

def match_format_reward(prompts, completions, **kwargs):
    """Reward for proper format usage."""
    scores = []
    for completion in completions:
        if match_format.search(completion):
            scores.append(2.0)
        elif REASONING_START in completion or SOLUTION_START in completion:
            scores.append(0.5)
        else:
            scores.append(-1.0)
    return scores

def rubric_overlap_score(response, rubric_text):
    """Calculate rubric overlap with TF-IDF weighting."""
    def tokenize(text):
        text = text.lower()
        for ch in string.punctuation:
            text = text.replace(ch, " ")
        return [t for t in text.split() if len(t) > 2]
    
    rubric_tokens = tokenize(rubric_text)
    response_tokens = set(tokenize(response))
    
    if not rubric_tokens:
        return 0.0
    
    token_counts = Counter(rubric_tokens)
    weighted_matches = sum(
        1.0 / token_counts[t] for t in response_tokens if t in token_counts
    )
    max_score = sum(1.0 / c for c in token_counts.values())
    
    coverage = weighted_matches / max_score if max_score > 0 else 0.0
    return coverage * 10.0

def rar_reward(prompts, completions, rubric=None, reference_response=None, **kwargs):
    """Rubric-as-Reward scoring."""
    rubrics = rubric or [""] * len(completions)
    references = reference_response or [""] * len(completions)
    
    rewards = []
    for response, rub, ref in zip(completions, rubrics, references):
        # Rubric overlap (0-10)
        r_score = rubric_overlap_score(response, rub) if rub else 0.0
        
        # Reference similarity (0-5)
        f_score = difflib.SequenceMatcher(None, ref, response).ratio() * 5.0 if ref else 0.0
        
        rewards.append(r_score + f_score)
    
    return rewards

print("Reward functions defined.")

## 6. Setup Training

In [ ]:
# Create sampler for generation
sampler = sampler_lib.Sampler(
    model=lora_policy,
    tokenizer=tokenizer,
    cache_config=sampler_lib.CacheConfig(
        num_layers=model_config.num_layers,
        num_kv_heads=model_config.num_kv_heads,
        head_dim=model_config.head_dim,
    ),
    mesh=mesh,
)
print("Sampler created.")

In [ ]:
# Training hyperparameters
MAX_STEPS = int(NUM_BATCHES * 0.94)  # With train fraction
WARMUP_STEPS = int(0.1 * MAX_STEPS)

# Optimizer with warmup + cosine decay
optimizer = optax.adamw(
    learning_rate=optax.schedules.warmup_cosine_decay_schedule(
        init_value=0.0,
        peak_value=LEARNING_RATE,
        warmup_steps=WARMUP_STEPS,
        decay_steps=MAX_STEPS,
        end_value=0.0,
    ),
    b1=0.9,
    b2=0.99,
    weight_decay=0.1,
)
optimizer = optax.chain(
    optax.clip_by_global_norm(max_norm=0.1),
    optimizer,
)

print(f"Max steps: {MAX_STEPS}, Warmup: {WARMUP_STEPS}")

In [ ]:
# GRPO configuration
grpo_config = GRPOConfig(
    num_generations=2,
    num_iterations=1,
    beta=0.08,
    epsilon=0.2,
)

# Cluster configuration
cluster_config = rl_cluster_lib.ClusterConfig(
    max_prompt_length=256,
    total_generation_steps=512,
)

# Data iterator config
data_iter_config = base_rollout.DataIteratorConfig(
    batch_size=2,
    num_batches=NUM_BATCHES,
)

print("GRPO config created.")

In [ ]:
# Create RL cluster
rl_cluster = rl_cluster_lib.RLCluster(
    config=cluster_config,
    reference=ref_model,
    tokenizer=tokenizer,
    mesh=mesh,
    sampler=sampler,
)

# Checkpoint options
checkpointing_options = ocp.CheckpointManagerOptions(
    save_interval_steps=100,
    max_to_keep=3,
)

# Metrics logger
metrics_logging_options = metrics_logger.MetricsLoggerOptions(
    log_dir="/tmp/tensorboard/grpo",
    flush_every_n_steps=20,
)

print("RL cluster created.")

In [ ]:
# Create GRPO trainer
reward_fns = [match_format_reward, rar_reward]

grpo_trainer = GRPOLearner(
    rl_cluster=rl_cluster,
    reward_fns=reward_fns,
    algo_config=grpo_config,
    optimizer=optimizer,
    ckpt_dir=CHECKPOINT_DIR,
    ckpt_options=checkpointing_options,
    metrics_logger_options=metrics_logging_options,
)

print("GRPO trainer created.")
print(f"Checkpoints will be saved to: {CHECKPOINT_DIR}")

## 7. Train!

In [ ]:
# Create data iterator
train_iter = train_dataset.batch(data_iter_config.batch_size)

print("="*60)
print("Starting GRPO Training")
print("="*60)
print(f"Batches: {NUM_BATCHES}")
print(f"Checkpoint dir: {CHECKPOINT_DIR}")
print("="*60)

In [ ]:
# Run training
grpo_trainer.train(
    policy=lora_policy,
    data_iterator=train_iter,
    data_iterator_config=data_iter_config,
)

print("\n" + "="*60)
print("Training complete!")
print("="*60)

## 8. Export Checkpoint for Local Use

In [ ]:
# Find latest checkpoint
import glob

ckpt_dirs = sorted(glob.glob(f"{CHECKPOINT_DIR}/actor/*/"))
if ckpt_dirs:
    latest_ckpt = ckpt_dirs[-1]
    print(f"Latest checkpoint: {latest_ckpt}")
else:
    print("No checkpoints found!")

In [ ]:
# Convert to HuggingFace format for local use
# This creates adapter files compatible with PEFT

EXPORT_DIR = f"{CHECKPOINT_DIR}/hf_lora"
os.makedirs(EXPORT_DIR, exist_ok=True)

# Save LoRA state
lora_state = nnx.state(lora_policy)

# Filter to only LoRA parameters
lora_params = {}
def extract_lora(path, value):
    path_str = ".".join(str(p) for p in path)
    if "lora" in path_str.lower():
        lora_params[path_str] = value

jax.tree_util.tree_map_with_path(extract_lora, lora_state)

print(f"Found {len(lora_params)} LoRA parameters")
print(f"\nCheckpoint saved to: {CHECKPOINT_DIR}")
print(f"\nTo use locally:")
print(f"1. Download the checkpoint folder from Google Drive")
print(f"2. Place in your local checkpoints/ directory")

In [ ]:
# Create a zip file for easy download
if SAVE_TO_DRIVE:
    !cd {CHECKPOINT_DIR} && zip -r checkpoint_export.zip actor/
    print(f"\nZipped checkpoint: {CHECKPOINT_DIR}/checkpoint_export.zip")
    print("Download this file from Google Drive and extract to checkpoints/")

## 9. Quick Test

In [ ]:
# Test the trained model
test_question = "A store sells apples for $2 each. If I buy 5 apples, how much do I spend?"
test_prompt = format_prompt(test_question)

print("Testing trained model...")
print(f"Question: {test_question}")
print()

response = sampler(
    [test_prompt],
    total_generation_steps=256,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
)[0]

print("Response:")
print(response)

---

## Done!

Your checkpoints are saved. To use them locally:

1. Download `checkpoint_export.zip` from Google Drive
2. Extract to your local `checkpoints/` folder
3. Run: `python demo/demo.py --checkpoint ./checkpoints/actor/<step>/model_params`